<a href="https://colab.research.google.com/github/woodword-0/ML-Projects/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import random
from scipy.stats import boxcox
random.seed(9000)

In [ ]:
df = pd.read_csv('/content/Churn_Modelling - Churn_Modelling.csv')

Outlier and Dropped Columns


In [ ]:
df = df[df['CreditScore'] > 400]  
df = df[df['Age'] <= 60]  
df = df[df['NumOfProducts'] < 4]  
df.drop(['RowNumber','CustomerId'],axis = 1)

Imputing and Value Types

In [ ]:
df['CustomerId'].astype('int')
vc_item1 = df['Surname'].value_counts()

df['Surname'] = df['Surname'].apply(lambda x: vc_item1[x])
df.loc[df['Geography'] == 'France', 'Geography'] = 2
df.loc[df['Geography'] == 'Germany', 'Geography'] = 1
df.loc[df['Geography'] == 'Spain', 'Geography'] = 0
df['Geography'] = df['Geography'].astype('int')
df.loc[df['Gender'] == 'Female', 'Gender'] = 0
df.loc[df['Gender'] == 'Male', 'Gender'] = 1
df['Gender'] = df['Gender'].astype('int')

df.drop(['RowNumber','CustomerId'],axis = 1, inplace= True)

In [ ]:
def minimax(s):
  df[s]=(df[s]-df[s].min())/(df[s].max()-df[s].min())


In [ ]:
minimax('Surname')
minimax('CreditScore')
minimax('Age')
minimax('Balance')
minimax('EstimatedSalary')

In [ ]:
ax = plt.figure(figsize=(8,8))
plt.scatter(x = df['CreditScore']	,y = df['Age'], c = df['Gender'] , alpha = 1,s = 2 ) #predict
# draw_circle3 = plt.Circle((0.83,0.59), 0.03, color = 'r', fill = False )
# ax.add_artist(draw_circle3)
plt.xlabel("CreditScore")
plt.ylabel('Age')
plt.title('CreditScore vs Age (Colored by Male Female)')
# plt.scatter(X,y, c = 'b', alpha = 0.5,s = 1000, edgecolors = '0' ) #predict
plt.show()

In [ ]:
# X_train = df_new.iloc[:11397,0:11].to_numpy() #goes to 11396
# y_train = df_new.iloc[:11397,11].to_numpy()
# X_validate = df_new.iloc[11397:12822,0:11].to_numpy()
# y_validate = df_new.iloc[11397:12822,11:12].to_numpy()
# X_test = df_new.iloc[12822:14246,0:11].to_numpy()
# y_test = df_new.iloc[12822:14246,11:12].to_numpy()
# X_train = df_new.iloc[:11397,0:11].to_numpy() #goes to 11396
# y_train = df_new.iloc[:11397,11].to_numpy()
# X_validate = df_new.iloc[11397:12822,0:11].to_numpy()
# y_validate = df_new.iloc[11397:12822,11:12].to_numpy()
# X_test = df_new.iloc[12822:14246,0:11].to_numpy()
# y_test = df_new.iloc[12822:14246,11:12].to_numpy()
data1 = df.drop("HasCrCard",axis=1) 
data2 = df['HasCrCard']
#match new indices
data1.reset_index(drop=True, inplace=True)
data2.reset_index(drop=True, inplace=True)
df_new = pd.concat([data1,data2], axis = 1)
df_new.drop(['Exited','NumOfProducts','CreditScore','Surname','Tenure','Balance','IsActiveMember'],axis = 1, inplace= True)

In [ ]:
X_train = df_new.iloc[:7568,0:4].to_numpy() #goes to 11396
y_train = df_new.iloc[:7568,4].to_numpy()
X_validate = df_new.iloc[7568:8514,0:4].to_numpy()
y_validate = df_new.iloc[7568:8514,4:5].to_numpy()
X_test = df_new.iloc[8514:9460,0:4].to_numpy()
y_test = df_new.iloc[8514:9460,4:5].to_numpy()

In [ ]:
plt.figure(figsize= (15,15))
corr=df_new.corr()
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
def linear(H):
  return H
  
def ReLU(H):
  return H*(H>0)

def sigmoid(H):
  return 1/(1+ np.exp(np.float128(-H)))

def softmax(H):
  eH=np.exp(H)
  return eH/eH.sum(axis=1, keepdims =True)

def cross_entropy(Y,P_hat):
  return -(1/len(Y))*np.sum(Y*np.log(P_hat))

def OLS(Y,Y_hat):
  return (1/(2*len(Y)))*np.sum((Y - Y_hat)**2)

def one_hot_encode(y):
  N=len(y)
  K = len(set(y))
  Y=np.zeros((N,K))

  for i in range(N):
    Y[i, y[i]]=1

  return Y

def accuracy(y, y_hat):
  return np.mean(y==y_hat)

def R2(y,y_hat):
  return 1- np.sum((y - y_hat)**2)/np.sum((y - y.mean())**2)
def derivative(Z,a):
  if a==linear:
    return 1
  elif a==sigmoid:
    return Z*(1-Z)
  elif a==np.tanh:
    return 1 - Z*Z
  elif a==ReLU:
    return (Z>0).astype(int)
  else:
    ValueError("Unknown Activation Function")

In [ ]:
class  ANN():

  def __init__(self,architecture, activations = None, mode = 1):
    self.mode = mode
    self.architecture = architecture
    self.activations = activations
    self.L = len(architecture) + 1

  def fit(self, X, y, eta = 1e-3, epochs = 1e3, show_curve = False):
    epochs = int(epochs)
    if self.mode:
      Y=one_hot_encode(y)
    else:
      Y = one_hot_encode(y)
    N,D = X.shape
    K = Y.shape[1]

    #Initialize weights and biases: Stochastic Initialization
    self.W = {l:np.random.randn(M[0],M[1]) for l,M in enumerate(zip(([D] + self.architecture), (self.architecture + [K])),1)}

    self.b = {l: np.random.randn(M) for l,M in enumerate(self.architecture + [K], 1)}

    #Activate Function Loading
    if self.activations is None:
      self.a = {l: ReLU for l in range(1, self.L)}
    else:
      self.a = {l: act for l,act in enumerate(self.activations, 1)}

    #Mode Set
    if self.mode:
      self.a[self.L] = linear
    else:
      self.a[self.L] = softmax #use sigmoid for binary
    J = np.zeros(epochs)

    #Gradient Descent/Back Propogation
    for epoch in range(epochs):
      self.forward(X)

      if self.mode:
        J[epoch] = OLS(Y,self.Z[self.L])
      else:
        J[epoch] = cross_entropy(Y,self.Z[self.L])

      dH = (1/N)*(self.Z[self.L]-Y)
      for l in sorted(self.W.keys(), reverse = True):
        dW = self.Z[l-1].T@dH
        db = dH.sum(axis = 0)

        #Update rules
        self.W[l] -= eta*dW
        self.b[l] -= eta*db

        if l > 1:
          dZ = dH@self.W[l].T
          dH = dZ*derivative(self.Z[l-1],self.a[l-1])

    if show_curve:
      plt.figure()
      plt.plot(J)
      plt.xlabel("epochs")
      plt.ylabel('$\mathcal{J}$')
      plt.title('Training Curve')
      plt.show()

  def forward(self,X):
    self.Z = {0:X}

    for l in sorted(self.W.keys()):
      self.Z[l] = self.a[l](self.Z[l-1]@self.W[l] + self.b[l])

  def predict(self, X):
    self.forward(X)

    if self.mode:
      return self.Z[self.L]
    else:
      return self.Z[self.L].argmax(axis = 1)



len(y_train)


In [ ]:
ann = ANN([16,12,16,2], [ReLU,ReLU, ReLU,sigmoid])
ann.fit(X_train, y_train, eta = 1e-2, epochs = 1000, show_curve = True)  #ann = ANN([16,12,16,2], [ReLU,ReLU, ReLU,sigmoid])
# ann.fit(X_train, y_train, eta = 1e-2, epochs = 500, show_curve = True)

In [ ]:
y_hat = ann.predict(X_train)

In [ ]:
y_hat = y_hat[:,0]

In [ ]:
def filters(x):
  if x > 0.2:
    return 1
  else:
    return 0

In [ ]:
preds = pd.DataFrame(y_hat)
preds = preds[0].apply(filters)
preds

In [ ]:
print(f"Training Accuracy: {accuracy(y_train, preds.to_numpy()):0.4f}")

In [ ]:
def confusion_matrix(y, y_hat):
  plt.figure(figsize=(10,7))
  y_actu = pd.Series(y, name='Actual')
  y_pred = pd.Series(y_hat, name='Predicted')
  cm = pd.crosstab(y_actu, y_pred)
  ax = sns.heatmap(cm, annot=True, fmt="d")
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  #plt.savefig("/content/drive/MyDrive/cm")

In [ ]:
confusion_matrix(y_train, preds.to_numpy())